# Init

In [5]:
import numpy as np
from utils import *
from skfem.helpers import *
from scipy.sparse.linalg import LinearOperator, minres, cg
from skfem.assembly import BilinearForm, LinearForm
import datetime
import pandas as pd
from skfem.visuals.matplotlib import draw, plot
import sys
import time

tol = 1e-8
intorder = 5
solver_type = 'mgcg'
refine_time = 5
epsilon_range = 6
element_type = 'P1'
sigma = 5
penalty = False
# epsilon = 1e-5
example = 'ex4'
save_path = 'log/' + example + '_' + element_type + '_' + ('pen' if penalty else 'nopen') + '_' +'{}'.format(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

In [6]:
def dirichlet(w):
    """return a harmonic function"""
    x, y = w
    theta = arctan3(y, x)
    return (x**2 + y**2)**(5/6) * sin(5*theta/3)

# def dirichlet(w):
#     """return a harmonic function"""
#     global out
#     x, y = w
#     out = np.zeros_like(x)

#     if x.ndim == 2:
#         for i in range(x.shape[0]):
#             for j in range(x.shape[1]):
#                 if x[i, j] != 0:
#                     out[i, j] = (x[i, j]**2 + y[i, j]**2)**(5 / 6) * sin(5 * atan(y[i, j] / x[i, j]) / 3)
#                 elif y[i, j] > 0:
#                     out[i, j] = (x[i, j]**2 + y[i, j]**2)**(5 / 6) * sin(pi/2 * 5 / 3)
#                 else:
#                     out[i, j] = (x[i, j]**2 + y[i, j]**2)**(5 / 6) * sin(3*pi/2 * 5 / 3)
#     elif x.ndim == 1:
#         for i in range(x.shape[0]):
#             if x[i] != 0:
#                 out[i] = (x[i]**2 + y[i]**2)**(5 / 6) * sin(5 * atan(y[i] / x[i]) / 3)
#             elif y[i] > 0:
#                 out[i] = (x[i]**2 + y[i]**2)**(5 / 6) * sin(pi/2 * 5 / 3)
#             else:
#                 out[i] = (x[i]**2 + y[i]**2)**(5 / 6) * sin(3*pi/2 * 5 / 3)
#     else:
#         raise Exception("dim not supported")
#     return out

# @LinearForm
# def boundary_load_un(v, w):
#     '''
#     u_n * v
#     '''
#     global x, www, vvv
#     vvv = v 
#     www = w
#     x, y = w.x
#     u_x, u_y = dexact_u(x, y)
#     return (w.n[0] * u_x + w.n[1] * u_y) * v

# @LinearForm
# def boundary_load_gradun(v, w):
#     '''
#     grad u_n * v
#     '''
#     x, y = w.x
#     u_xx, u_xy, u_yx, u_yy = ddexact(x, y)
#     return (u_xx * w.n[0] + u_xy * w.n[1]) * grad(v)[0] + (u_yx * w.n[0] + u_yy * w.n[1]) * grad(v)[1]

In [37]:
def solve_problem1(m, element_type='P1', solver_type='pcg', intorder=6, tol=1e-8, epsilon=1e-6):
    '''
    switching to mgcg solver for problem 1
    '''
    if element_type == 'P1':
        element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
    elif element_type == 'P2':
        element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
    else:
        raise Exception("Element not supported")

    basis = {
        variable: InteriorBasis(m, e, intorder=intorder)
        for variable, e in element.items()
    }
    
    K1 = asm(laplace, basis['w'])
    f1 = asm(f_load, basis['w'])
    wh = solve(*condense(K1, f1, D=basis['w'].find_dofs()), solver=solver_iter_mgcg(tol=tol))
    
    global K2, f2, uh0, boundary_dofs, boundary_basis, boundary_dofs_un
    fbasis = FacetBasis(m, ElementTriMorley())
    K2 = epsilon**2 * asm(a_load, basis['u']) + asm(b_load, basis['u'])
#     f2 = asm(wv_load, basis['w'], basis['u']) * wh + asm(boundary_load_un, fbasis) + epsilon**2 * asm(boundary_load_gradun, fbasis)
    f2 = asm(wv_load, basis['w'], basis['u']) * wh
#     boundary_basis = FacetBasis(m, element['w'])
    boundary_basis = FacetBasis(m, ElementTriMorley())
#     boundary_dofs = boundary_basis.find_dofs()
    boundary_dofs = boundary_basis.find_dofs()['all'].all()
#     boundary_dofs_u = basis['u'].nodal_dofs[0]
#     boundary_dofs = np.array([i for i in basis['u'].find_dofs()['all'].all() if i in basis['u'].nodal_dofs[0]])
    boundary_dofs_u = np.array([i for i in boundary_dofs if i in basis['u'].nodal_dofs[0]])
    boundary_dofs_un = np.array([i for i in boundary_dofs if i in basis['u'].facet_dofs[0]])
    print(boundary_dofs_u)
    print(boundary_dofs_un)
#     boundary_dofs_un = basis['u'].facet_dofs[0]
#     boundary_dofs = m.boundary_nodes()
#     boundary_dofs = easy_boundary(m, basis['u'])
    uh0 = np.zeros(basis['u'].N)
    print(boundary_dofs)
#     print(basis['u'].N)

#     uh0[boundary_dofs] = pproject(dirichlet, basis_to=boundary_basis, I=boundary_dofs, solver=minres)
    uh0[boundary_dofs_u] = exact_u(boundary_basis.doflocs[0][boundary_dofs_u], boundary_basis.doflocs[1][boundary_dofs_u])
    uh0[boundary_dofs_un] = exact_un(boundary_basis.doflocs[0][boundary_dofs_un], boundary_basis.doflocs[1][boundary_dofs_un])
    #     boundary_dofs = m.boundary_nodes()
#     print(uh0[boundary_dofs_un])
#     temp = uh0
#     boundary_dofs = np.hstack([boundary_dofs, easy_boundary(m, basis['u'])])
#     print(exact_u(boundary_basis.doflocs[0][boundary_dofs], boundary_basis.doflocs[1][boundary_dofs]))
    print(uh0)
#     uh0 = solve(*condense(K2, f2, D=easy_boundary(m, basis['u'])), solver=solver_iter_mgcg(tol=tol))
#     uh0 = solve(*condense(K2, f2, D=basis['u'].find_dofs()), solver=solver_iter_mgcg(tol=tol))
#     uh0 = solve(*condense(K2, f2, uh0, D=boundary_dofs), solver=solver_iter_krylov(minres, tol=tol))
#     tol = 1e-1
    uh0 = solve(*condense(K2, f2, uh0, D=boundary_dofs), solver=solver_iter_mgcg(tol=tol))
#     uh0 = solve(*condense(K2, f2, uh0, D=basis['u'].find_dofs()), solver=solver_iter_mgcg(tol=tol))
#     print(uh0)
#     print(uh0-temp)
    return uh0, basis

# Solving

In [38]:
x = boundary_basis.doflocs[0][boundary_dofs_un]
y = boundary_basis.doflocs[1][boundary_dofs_un]

In [39]:
def exact_un(x, y):
    nx = -1 * (x == -1) + 1 * ((x == 1) + (x == 0) * (y > 0))
    ny = -1 * (y == -1) + 1 * ((y == 1) + (y == 0) * (x > 0))
#     nx = -1 * (x == 0) + 1 * (x == 1)
#     ny = -1 * (y == 0) + 1 * (y == 1)
#     print(nx)
#     print(ny)
    dux, duy = dexact_u(x, y)
    out = nx * dux + ny * duy
#     out[np.isnan(out)] = 0
    return out

In [40]:
u

array([0.        , 0.5       , 0.        , 1.72108416, 0.15749013,
       0.        , 0.54210754, 1.159291  , 0.84077245, 0.04960628,
       0.        , 0.30955568, 0.58959925, 0.8443497 , 0.        ,
       0.34534261, 0.41762199, 1.45003215, 1.27411462, 0.17075318,
       0.36515378, 0.26482673, 0.83921196, 0.6985417 , 1.06554275,
       0.015625  , 0.        , 0.40023656, 0.56088129, 0.67618834,
       0.        , 0.19073244, 0.20845361, 1.58777283, 1.4957281 ,
       0.09750393, 0.22843889, 0.18571213, 0.26595349, 0.        ,
       0.        , 0.10877611, 0.13154268, 0.58398988, 0.46266461,
       0.45673151, 0.40132096, 0.68791595, 0.62241818, 1.00516004,
       1.30738386, 0.9962813 , 1.12850593, 0.62818723, 1.05592577,
       0.77115252, 0.96972605, 0.05378388, 0.11501623, 0.08341519,
       0.45387108, 0.35545335, 0.71323428, 0.47355358, 0.71773023,
       0.85810377, 0.17215155, 0.15251682, 0.3823159 , 0.30634751,
       0.52053178, 0.57555143, 1.37768162, 1.25544388, 1.17176

In [41]:
uh0

array([ 0.00000000e+00,  5.00000000e-01,  0.00000000e+00,  1.72108416e+00,
        1.57490131e-01,  0.00000000e+00,  4.82182039e-01,  1.15929100e+00,
        8.40772452e-01,  4.96062829e-02,  0.00000000e+00,  3.09555680e-01,
        5.51517644e-01,  8.44349696e-01,  0.00000000e+00,  3.05685919e-01,
        4.17621992e-01,  1.45003215e+00,  1.27411462e+00,  1.32264228e-01,
        3.17221233e-01,  2.15748561e-01,  7.95370594e-01,  6.53968025e-01,
        1.03470342e+00, -3.60843918e-01, -4.16666667e-01, -1.32043821e+00,
        6.56860381e-17,  1.26436761e+00, -1.52471071e+00, -3.58721528e-16,
        5.45426316e-01,  1.29943501e+00,  1.89497404e+00, -7.50585597e-01,
       -1.05511402e+00,  4.98657023e-16,  3.42481772e-16, -8.66701593e-01,
       -1.21834072e+00,  6.64361552e-17,  1.82010959e-16, -7.88002050e-16,
        7.41651251e-16, -8.79873864e-16,  8.34558197e-17, -4.98451220e-16,
        1.00909928e-15,  1.73426880e+00,  1.68121193e+00,  4.67213948e-16,
        3.81459479e-16,  

In [ ]:
# m = MeshTri().init_lshaped()
m = MeshTri()
m.refine(2)
# draw(m)

epsilon = 0
ep = epsilon

uh0, basis = solve_problem1(m, element_type, solver_type, intorder, tol, epsilon)

x = basis['u'].doflocs[0]
y = basis['u'].doflocs[1]
u = exact_u(x, y)
plot(basis['u'], u-uh0, colorbar=True)
# plot(basis['u'], u, colorbar=True)

In [ ]:
# m = MeshTri().init_lshaped()
m = MeshTri()
m.refine(4)
# draw(m)

epsilon = 0
ep = epsilon

uh0, basis = solve_problem1(m, element_type, solver_type, intorder, tol, epsilon)

x = basis['u'].doflocs[0]
y = basis['u'].doflocs[1]
u = exact_u(x, y)
plot(basis['u'], u-uh0, colorbar=True)
# plot(basis['u'], u, colorbar=True)

In [ ]:
basis['u'].tind

In [ ]:
# basis['u'].doflocs

# # basis['u'].nodal_dofs[0]
# for i in range(25):
# #     print(i)
# #     print(boundary_basis.doflocs[:, i])
#     pass

# basis['u'].find_dofs()['all'].all()

# np.array([i for i in basis['u'].find_dofs()['all'].all() if i in basis['u'].nodal_dofs[0]])

# basis['u'].facet_dofs[0]

# uh0


# np.arctan2(1, 0)

from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib import pyplot as plt
fig = plt.figure()
ax = Axes3D(fig)
x = np.arange(-1, 1, 0.05)
y = np.arange(-1, 1, 0.05)
X, Y = np.meshgrid(x, y) 
Z = exact_u(X, Y)
plt.xlabel('x')
plt.ylabel('y')

ax.plot_surface(X, Y, Z, rstride=2, cstride=1, cmap='rainbow')
plt.show()

# boundary_dofs

# easy_boundary(m, basis['u'])
# np.hstack([boundary_dofs, easy_boundary(m, basis['u'])])

In [ ]:
boundary_basis.find_dofs()['all'].all()

In [ ]:
# v.value

In [ ]:
# (w.n[0] * u_x + w.n[1] * u_y) * v

In [ ]:
boundary_basis.find_dofs()['all'].all()

In [ ]:
basis['u'].nodal_dofs[0]

In [ ]:
basis['u'].facet_dofs[0]

In [ ]:
uh0.shape

boundary_dofs.shape

basis['u'].facet_dofs

basis['u'].nodal_dofs[0]

basis['u'].get_dofs()

basis['u'].find_dofs()['all'].all()

pproject(dirichlet, basis_to=boundary_basis, I=boundary_dofs, solver=minres)

exact_u(boundary_basis.doflocs[0][boundary_dofs], boundary_basis.doflocs[1][boundary_dofs])

boundary_dofs

In [ ]:
plot(basis['u'], u, colorbar=True)

In [ ]:
plot(basis['u'], uh0, colorbar=True)

In [ ]:
# w = www
# v = vvv
# x, y = w.x
# u_x, u_y = dexact_u(x, y)
# (w.n[0] * u_x + w.n[1] * u_y) * v

In [ ]:
exact_u(x, y)

In [ ]:
dirichlet([x, y])

In [ ]:
exact_u(np.array([-0.00000]), np.array([0.8]))

In [ ]:
dirichlet([np.array([[-0.00000]]), np.array([[0.8]])])

In [ ]:
boundary_basis.mesh

In [ ]:
exact_u(boundary_basis.doflocs[0], boundary_basis.doflocs[1])

In [ ]:
pproject(dirichlet, basis_to=boundary_basis, I=boundary_dofs, solver=minres)

In [ ]:
(uh0 - u)[:30]

In [ ]:
u

In [ ]:
time_start = time.time()

print('=======Arguments=======')
print('penalty:\t{}'.format(penalty))
print('element_type:\t{}'.format(element_type))
print('solver_type:\t{}'.format(solver_type))
print('tol:\t{}'.format(tol))
print('intorder:\t{}'.format(intorder))
print('refine_time:\t{}'.format(refine_time))
print('sigma:\t{}'.format(sigma))
print('=======Results=======')

df_list = []
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j)
    ep = epsilon
    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
#     m = MeshTri().init_lshaped()
    m = MeshTri()
#     draw(m)

    for i in range(1, refine_time+1):
        
        m.refine()
        
        if penalty:
            uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder, tol, epsilon)
        else:
            uh0, basis = solve_problem1(m, element_type, solver_type, intorder, tol, epsilon)

        U = basis['u'].interpolate(uh0).value

        # compute errors

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        if penalty:
            D2u = np.sqrt(get_D2uError(basis['u'], uh0)**2 + L2pnvError.assemble(fbasis, w=fbasis.interpolate(uh0)))
        else:
            D2u = get_D2uError(basis['u'], uh0)
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)
        
    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us
    
    # store data
    data = np.array([L2s, H1s, H2s, epus])
    df = pd.DataFrame(data.T, columns=['L2', 'H1', 'H2', 'Energy'])
    df_list.append(df)
    
    print('epsilion:', epsilon)
    show_result(L2s, H1s, H2s, epus)

time_end = time.time()

result = df_list[0].append(df_list[1:])
result.to_csv(save_path+'.csv')
print('======= Errors saved in:', save_path+'.csv ==========')
print('Total Time Cost {:.2f} s'.format(time_end-time_start))